# Mixed Integer Linear Programming approach for Plasmid Assembly

#### Aniket Mane, Mahsa Faizrahnemoon, Cedric Chauve
#### Department of Mathematics, Simon Fraser University

### Abstract
This notebook describes a Mixed Integer Linear Programming (MILP) approach for extracting plasmids from Whole Genome Sequencing (WGS) data. We use this tool for assembling plasmids from a data set of 133 bacterial samples.

In [2]:
from datetime import date
print(f"Version of {date.today()}.")

Version of 2021-06-23.


## Introduction

Antibiotic resistance is developed by bacteria through the spread of antimicrobial genes. This is often facilitated by mobile genetic elements. The family of plasmids falls under the umbrella of mobile genetic elements. Plasmids are circular, double-stranded segments of DNA that are separate from chromosomes. They are important due to their role in the spread of antibiotic resistance through mechanisms such as horizontal gene transfer. Thus, the characterization and assembly of plasmids is vital for understanding their
evolution and transmission.

Various tools have been designed for predicting and assembling of plasmids from Illumina short-read contigs as input. This problem can be studied at three different levels: identifying plasmidic contigs from chromosomal ones, grouping contigs from the same plasmid and assembling the grouped contigs into full plasmids. Our method aims at inferring plasmids at the assembly stage.

## Preliminaries

The main input for plasmid assembly tools is the *assembly graph*. The vertices of this graph represent *contigs*, continuous sequences of overlapping DNA segments. Every contig has two *extremeties*, a head and a tail. Thus, a contig $c$ will have two extremeties, $c_h$ and $c_t$. The edges or *links* of the assembly graph represent pairs of extremeties, the contigs of which might be next to each other in a plasmid. A link between $c_h$ and $d_h$ will be represented as $(c_h, d_h)$.

As plasmids are typically circular in nature, a plasmid assembly tools usually try to identify cycles
of contigs from the assembly graph that represent putative plasmids. In order to decide if a cycle (or path in case of some tools) predicted by a tool is a plasmid, we look at the contigs in the cycle or path. Each contig has some features associated with it, namely, 
- gene density (proportion of the sequence containing plasmid genes)
- read depth (the average number of times a segment in the contig has been sequenced)
- GC content (proprtion of the sequence containing G or C bases)
- length (the length of the sequence) 

Furthermore, based on these features, we designate specific contigs as *seed* contigs. Such contigs are considered plasmidic and are expected to appear as part of some plasmid in the solution. Thus, an ideal solution representing a plasmid would consist of a sequence of contigs that display certain characteristics such as having high plasmid gene density. We describe the method and the rationale behind it in the upcoming sections. 

## Method

### Input and output
We propose a MILP to extract plasmids from a given assembly graph. The objective function for the MILP quantifies the features that characterize a plasmid. Thus, we assume that we have the information about the features of individual contigs. 

We use the following as input:
1. Assembly graph (vertices and edges),
2. Length of contig ( len[c] )
3. Gene coverage of contig ( gd[c] )
4. Read depth of each contig ( rd[c] )
5. GC content of each contig ( GC[c] )
6. Seed eligibility of a contig (seed[c] )

Each iteration of the MILP outputs one plasmid. The assembly graph is then modified to reflect the removal of this plasmid. The MILP continues till no more plasmids can be extracted. 

We expect the following as output:
1. List of contigs belonging to a plasmid,
2. List of edges/links belonging to a plasmid

## Motivation for objective function and constraints

Our objective function consists of three terms. Here, we provide the reasoning behind eeach term.

- Read depth difference: Firstly, we try to obtain uniform read depth along the plasmid. We try to minimize the difference between the read depth of the overall plasmid and the read depth contribution of the contig itself. Furthermore, as each plasmid may have different total length, we consider the wieghted read depth deviation. Thus, rd[p][c] is the read depth contribution of a contig to a plasmid, len[c] is the length of a contig and meanrd[p] is the mean read depth of the plasmid, the expression we try to minimize is: $\sum_{c \in p}$(|mean rd[p] - rd[p][c]|).len[c]/len[p]. 

- %GC content difference: The %GC content of plasmids is slightly lower than that of chromosomes. Keeping this criterion in mind, the objective would be to maximize the difference between the %GC content of the plasmid and the overall %GC content. Here, we represent the %GC content of a contig as GC[c] and GC mean as the overall %GC content. This is a constant and can be computed in advance. NOTE: As the final objective function expression will be a linear combination of multiple terms, it is advisable for these terms to be of comparable values. Having a term account for most of the objective function value may lead to the term dominating others. Hence, we again weight the %GC content difference according by the length of the contig. $\sum_{c \in p}$(GCmean - GC[c]).len[c]/len[p].

- Gene coverage: We also try to maximize the gene coverage or density, which is the percentage of plasmids covered by genes. The gene density of a contig is represented as gd[c], which is a number between 0 and 1. Once again, we weight the gene density of selected contigs by their lengths. $\sum_{c \in p}$(gd[c]).len[c]/len[p].

### Objective function

Thus, our objective is to minimize: 

$\alpha_1 .\sum_{c \in p}$[ ($|$meanrd[p] - rd[p][c]$|$).len[c] $/\sum_{c \in p}$len[c]\} ]

$- \alpha_2 .\sum_{c \in p}$[ (GCmean - GC[c]).len[c]) $/\sum_{c \in p}$len[c]\} ] 

$- \alpha_3 .\sum_{c \in p}$[ gd[c].len[c] $/\sum_{c \in p}$len[c]\} ] 

where $\alpha_1=\alpha_2=\alpha_3 = 1$

Note that the values of the $\alpha_i$'s can be chosen based on the importance you wish to assign to a particular term in the objective function. 

### Constraints

A large set of constraints help us in modelling the problem. Most of the constraints enable the MILP to extract components from the assembly graph that are either paths or trails.

We also separate questionable plasmids away from putative plasmids. The conditions under which a plasmid is classified as putative are as follows:
- The gene density of a plasmid has a prespecified minimum threshold. This threshold is decided by computing the the average gene density of plasmids in the data set.
- The length of a plasmid has a maximum and minimum threshold. This is done particularly to avoid chromosomal contigs in the solution plasmid.

We require that the plasmid is a single connected component. To achieve this, we can use degree constraints (the number of edges is one less than (equal to in case of a cyclic components) the number of vertices. However, the MILP will not able to reject solutions that have multiple other cycles in addition to the main path or cycle. To address this issue, we also add constraints to prevent cycles. Hence, we choose to extract a single path from the graph instead of a cycle. Thus, after each iteration of the MILP, a plasmid is output, it is checked for cyclic components and constraints are added to prevent those cycles. The MILP is then rerun.  

### Modifications performed to simplify the problem

Initial experiments performed using this strategy provided us with some insight on the behaviour of the MILP. 

- The normalization of the terms in the objective function by the total length of the contigs in the plasmid led to MILP selecting the singular best contig in the assembly graph. Appending a neighbouring contig would simply worsen the objective function value. Thus, for the subsequent formulations, we chose to remove the normalization term. Note that we still use lengths of contigs as the weights of the terms. This ensures that the contribution of a longer contig counts more towards the objective function than a shorter one.

- Some plasmids contain copies of the same contig in multiple places. Due to the cycle prevention constraints, this prevented the prediction of such plasmids. In order to allow for multiple copies of the contig to occur in the same plasmid, we modified the assembly graph as follows: Each contig $c$ of the graph has a read depth $k$ associated with it. We split the contig into $k$ copies $c_1, c_2, ..., c_k$. Any link incident on $c$ in the original graph is incident on every $c_i$ in the modified graph. Each copy $c_i$ has read depth 1. The other attributes (gene density, GC content and length) remain the same as for the original contig $c$. Since each vertex in the modified assembly graph has read depth uniformly 1, this rids the necessity of the read depth deviation term in the objective function.

Thus, the new objective function is   

$- \alpha_2 .\sum_{c \in p}$[ (GCmean - GC[c]).len[c]) ] 

$- \alpha_3 .\sum_{c \in p}$[ gd[c].len[c] ] 

where $\alpha_2=\alpha_3 = 1$

- In some samples, it is possible that a contig has a very high read depth. In the modified graph, this resulted in a contig with a high number of vertex copies in the graph. Thus if a group of contigs with high read depths form a cycle, the MILP would allow for multiple copies of the same cycle, using different copies of the same group of vertices. In order to avoid such repetitive cycles, we added edge novelty constraints. For instance, if an edge $(a_{1h}, b_{2t})$ between has already been used (links[p][a_{1h}, b_{2t}]==1), then any other edge $(a_{ih}, b_{jt})$ will not be part of the solution. Note that by using this constraint, we also prevent a solution from having repeated edges that may not be part of repetitive cycles. 

### Nonlinear constraints

- Handling ’if ’ conditions: 
Multiple terms in the objective function depend on the inclusion of a contig c in plasmid p. For instance, the length of a contig should be "counted" towards the length of a plasmid only if it is a part of the plasmid. So, we introduce the term countedlen[p][c]. 
countedlen[p][c] == len[p][c].contigs[p][c]
This is a nonlinear constraint. However, since contigs[p][c] is a binary variable and len[p][c] has a well-defined upper limit (len[c]), we can model this using a set of linear constraints. Here U = len[c].

countedlen[p][c] $<=$ U . contigs[p][c] <br>
countedlen[p][c] $<=$ len[p][c] <br>
countedlen[p][c] $>=$ len[p][c] - (1 - contigs[p][c]).U <br>
countedlen[p][c] $>=$ 0

Notice that counted len[p][c] takes exactly the value len[p][c] if contigs[p][c] = 1 and 0 otherwise.

## Current progress in the experiments

We experimented with various versions of the MILP, making modifications to be improve its performance. Initial versions of the MILP did not yield good results in terms of precision and recall. However, we are hopeful that the current version of the MILP improves the performance considerably.

The current MILP outputs one plasmid per iteration. In other words, although we remove one plasmid at a time in this experiment. The MILP however, has provisions to remove $k>1$ plasmids in one iteration, provided convergence is reached. The main motivation behind the iterative plasmid generation approach is to help in reaching convergence by reducing the number of variables.

A set of 10 samples was chosen to test the general performance of the MILP. These were the same samples as the ones chosen to test the greedy algorithm, [HyAsP](https://github.com/cchauve/HyAsP). The results for the 10 chosen samples are compared with the results of the greedy heuristic. 

The MILP is run using Gurobi solver with a maximum computation time of 3 hours (to be increased to 5 hours) and optimality gap of 0 (to be increased to 5%) 

Of the 10 samples, 6 samples reached the solution using the ILP. The MILP misses contigs from the true solution only in 1 of the 6 instances. In the specific case, the missing contig happens to have no plasmid genes (according to input data). Hence, the ILP is justified to have left it out. In 2 of the 6 samples, the extra contigs
predicted by the ILP have non-negigible length. In both cases, the precision drops below 0.75 as
a result. We observe 100% recall in 5 out of 6 cases.

## Next steps

Owing to promising results in from the last set of experiments, the aim is to perform experiments with all the 133 samples. We also wish to compare the performance of our method against currently used plasmid assembly methods such as [plasmidSPAdes](https://academic.oup.com/bioinformatics/article/32/22/3380/2525610), [Recycler](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5408804/) and [MOB-suite](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6159552/) along with HyAsP.